# Preparation

In [47]:
!pip install python-binance
!pip install pandas
!pip install tabulate
!pip install requests_oauthlib

  Using cached python_binance-1.0.15-py2.py3-none-any.whl (63 kB)
  Using cached aiohttp-3.8.1-cp310-cp310-win_amd64.whl (555 kB)
You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.
  Using cached pandas-1.3.5-cp310-cp310-win_amd64.whl (10.2 MB)
You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
!sudo apt-get install python3-dev libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
!sudo pip3 install -U sql_magic > /dev/null
!pip install psycopg2-binary > /dev/null

The system cannot find the path specified.
The system cannot find the path specified.
The system cannot find the path specified.
The system cannot find the path specified.


In [57]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [58]:
import requests
import json
from datetime import datetime
from tabulate import tabulate

In [59]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy import event

In [60]:

from requests_oauthlib import OAuth1
from pandas.io.json import json_normalize
from datetime import datetime, timedelta
from pytz import timezone

# Function

### Cryptocurrency Price

In [61]:
validIntervals = ['1m', '3m', '5m', '15m', '30m', '1h', '2h', '4h', '6h', '8h', '12h', '1d', '3d', '1w', '1M']

def searchBinance(symbols, startTime='2020-01-01 00:00:00', endTime='2020-12-02 00:00:00', interval='1M'):
    if interval not in validIntervals: 
        raise ValueError('Please enter a valid interval: ' + str(validIntervals))
    dfs = []
    url = 'https://api.binance.com/api/v3/klines'
    for symbol in symbols:
        try:
            params = {
                'symbol': symbol,
                'interval': interval,
                'limit': 1000,
              'startTime': int(datetime.strptime(startTime, '%Y-%m-%d %H:%M:%S').timestamp() * 1000),
                'endTime': int(datetime.strptime(endTime, '%Y-%m-%d %H:%M:%S').timestamp() * 1000)
            }
            response = requests.get(url, params=params)
            df = pd.DataFrame(response.json())
            df.columns = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
            df['Coin Pair'] = symbol
            df['Open time'] = df['Open time'].apply(lambda x: datetime.fromtimestamp(x/1000.0))
            df['Close time'] = df['Close time'].apply(lambda x: datetime.fromtimestamp(x/1000.0))
            # dfs.append(df[['Coin Pair', 'Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time']])
            dfs.append(df[['Coin Pair', 'Open time', 'Open', 'Close']])
        except:
            continue
    try:
        return pd.concat(dfs)
    except:
        return None


### Data Analysis

In [62]:
def cleanNewsData(coin_news_df):
  coin_news_df_raw = coin_news_df
  coin_news_df_sorted = coin_news_df_raw.sort_values(by=['date_published']).dropna()
  coin_news_df_sorted.drop(coin_news_df_sorted[coin_news_df_sorted['anger'] == 'error'].index, inplace = True)
  coin_news_df = coin_news_df_sorted.reset_index().drop(columns=['index'])

  for i in range(len(coin_news_df)):
    coin_news_df['date_published'][i] = str(coin_news_df['date_published'][i])[:10]
  coin_news_df = coin_news_df.drop(columns='url').rename(columns={'date_published':'Date','digust':'disgust'})

  return coin_news_df


In [63]:
def cleanTwitterData(coin_twitter_df):
  coin_twitter_df_raw = coin_twitter_df
  coin_twitter_df = coin_twitter_df_raw.dropna().reset_index()

  for i in range(len(coin_twitter_df)):
    coin_twitter_df['created_at'][i] = str(coin_twitter_df['created_at'][i])[:10]
  coin_twitter_df = coin_twitter_df.rename(columns={'created_at':'Date', 'digust':'disgust'})
  return coin_twitter_df 


In [64]:
def getData(twitter_or_news, coin_symbol):
  conn_string = 'mysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
      user="noerrors",
      password="JXEf1zCCp5c=",
      host="jsedocc7.scrc.nyu.edu",
      port=3306,
      db="NoErrors",
      encoding="utf-8"
  )
  engine = create_engine(conn_string)  
 
  if twitter_or_news == 'twitter':
    query = '''
    SELECT * FROM twitter_sentiment
    '''
    twitter_df = pd.read_sql(query, con=engine)
    coin_twitter_df = twitter_df[(twitter_df['coin_name']== coin_symbol)]
    coin_data_df = cleanTwitterData(coin_twitter_df)

  elif twitter_or_news == 'news':
    query = '''
    SELECT * FROM news_sentiment
    '''
    news_df = pd.read_sql(query, con=engine)
    coin_news_df = news_df[(news_df['coin_name']== coin_symbol)]  
    coin_data_df = cleanNewsData(coin_news_df)  

  else:
    print('Please enter twitter or news for the first argument')
  
  return coin_data_df

testcoin_data_df = getData('twitter', 'DOGE')
testcoin_data_df

,index,sentiment_id,coin_name,Date,author_id,anger,disgust,fear,joy,sadness,label,score
0,7549,7550,DOGE,2021-12-11,2435986338,0.049211,0.035718,0.062240,0.082141,0.153572,neutral,0.000000
1,7550,7551,DOGE,2021-12-11,1466339889155239943,0.004049,0.530035,0.017824,0.343809,0.241921,neutral,0.000000
2,7551,7552,DOGE,2021-12-11,980962034421587969,0.004049,0.530035,0.017824,0.343809,0.241921,neutral,0.000000
3,7552,7553,DOGE,2021-12-11,1369496417073438728,0.478327,0.059836,0.320713,0.255852,0.112219,negative,-0.720571
4,7553,7554,DOGE,2021-12-11,1267197215635726336,0.017612,0.014079,0.039293,0.316694,0.093051,neutral,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
678,8227,8228,DOGE,2021-12-17,1407030581372858377,0.012346,0.159198,0.078144,0.636822,0.086213,positive,0.843416
679,8228,8229,DOGE,2021-12-17,1452276873187274763,0.083894,0.020413,0.016207,0.401257,0.229325,negative,-0.393949
680,8229,8230,DOGE,2021-12-17,1246503261730938880,0.241229,0.195034,0.061266,0.382953,0.199295,negative,-0.751249
681,8230,8231,DOGE,2021-12-17,1454604514623954950,0.019555,0.270080,0.036131,0.501483,0.169975,positive,0.921830


In [77]:
def getPriceData(coin_news_df, coin_name_binance):
  start_time = str(coin_news_df['Date'][0])# + ' 00:00:00'
  end_time = str(coin_news_df['Date'][len(coin_news_df)-1])# + ' 00:00:00'
  coin_price_df_raw = searchBinance([coin_name_binance], startTime= start_time, endTime= end_time, interval='1d')
  coin_price_df = coin_price_df_raw.drop(columns=['Coin Pair']).rename(columns={'Open time':'Date', 'Close':'Price'})
  return coin_price_df


In [66]:
def combineSentPrice(coin_news_df, coin_price_df):
  coin_price_news_df = coin_news_df.copy()
  coin_price_news_df['Price'] = np.nan

  coin_news_df['Date'] = pd.to_datetime(coin_news_df['Date'])

  for i in range(len(coin_news_df)):
    price_row = coin_price_df.loc[coin_price_df['Date'] == coin_news_df['Date'][i]]
    try:
      price_index = list(price_row.to_dict()['Price'].keys())[0]
      price = coin_price_df['Price'][price_index]
      coin_price_news_df.at[i, 'Price'] = price
    except:
      continue

  coin_price_news_df.dropna(subset=['Price'], inplace=True)

  return coin_price_news_df


In [67]:
def dataAnalysis(coin_price_news_df):

  coin_price_news_df['anger'] = pd.to_numeric(coin_price_news_df['anger'])
  coin_price_news_df['disgust'] = pd.to_numeric(coin_price_news_df['disgust'])
  coin_price_news_df['fear'] = pd.to_numeric(coin_price_news_df['fear'])
  coin_price_news_df['joy'] = pd.to_numeric(coin_price_news_df['joy'])
  coin_price_news_df['sadness'] = pd.to_numeric(coin_price_news_df['sadness'])

  X = coin_price_news_df[["anger", "disgust", "fear", "joy", 'sadness']]
  y = coin_price_news_df.Price
  lr = LinearRegression()
  lr.fit(X, y)

  coefficient_intercept = np.append(lr.coef_, lr.intercept_)
  coin_price_news_df['intercept'] = np.nan
  format_dict = {'anger':[], 'disgust': [], 'fear':[], 'joy':[], 'sadness':[], 'intercept':[]}
  format_df = pd.DataFrame(format_dict)

  regression_df = pd.DataFrame(zip(format_df.columns[0:], coefficient_intercept), columns = ["sentiments", "coefficients"])
  regression_df

  return lr


### Get twitter info today

In [68]:
consumer_key = '2GEDtzlFMJK6agAMkPQoVTwnl'
consumer_secret = '9TvdpLsvdZDbUrihxDd2LUh02P3moWewdAWqTeupJH90SxPkoi'
access_token = '1241443545975791617-Qy2ioSjn5qmKfHN17bSKV1RhWv19et'
access_secret = 'asmAhYweQTDavRPwrs3FkdJd3557g76rcyksDNGT3b9Nx'

auth = OAuth1(consumer_key, consumer_secret, access_token, access_secret)

In [69]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAANNrMgEAAAAAauW1AUNHTT0LpndAup%2FVN3XJq6U%3Dj1Xk8EJ53dd7Lqwuzjhre9SGvuFebeJtitOOphYKhawST8keZU'

In [70]:
def getSentimentTwitter(text):
    endpoint = 'https://api.us-east.natural-language-understanding.watson.cloud.ibm.com/instances/d1c5c268-9db4-43a1-aa7d-b8ecaf566cc2/v1/analyze'
    username = "apikey"
    password = "Awa31seHtH1zVbGgt_cPK0lJkCHIqJIHsxaQBMqBEmKK"
    parameters = {
        'features': 'emotion,sentiment',
        'version' : '2021-08-01',
        'text': text,
        'language' : 'en',
    }
    resp = requests.get(endpoint, params=parameters, auth=(username, password))
    if resp.status_code != 400:
      emotion = resp.json()['emotion']['document']['emotion']
      sentiment = resp.json()['sentiment']['document']
      emotion.update(sentiment)
      return emotion
    else:
      return None

In [71]:
def getTwitterSentToday(coin):

  tz = timezone('EST')
  current_time = datetime.now(tz)
  formatted_current_time = current_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")

  yesterday = current_time - timedelta(days=1)
  yesterday.strftime("%Y-%m-%dT%H:%M:%S.000Z")
  formatted_yesterday_time = yesterday.strftime("%Y-%m-%dT%H:%M:%S.000Z")

  twitter_search_endpoint = 'https://api.twitter.com/2/tweets/search/recent'
  parameters  = {
    'query' : coin,
    'tweet.fields' : 'text,author_id,created_at,public_metrics',
    'start_time' : formatted_yesterday_time,
    'end_time' : formatted_current_time,
    'max_results' : 100
}

  res = requests.get(twitter_search_endpoint, auth=auth, params=parameters)

  tweets = res.json()
  tweets = tweets['data']

  date = []
  author = []
  text = []
  for i in range(len(tweets)):
    date.append(tweets[i]['created_at'])
    author.append(tweets[i]['author_id'])
    text.append(tweets[i]['text'])

  full_article = []
  
  for i in range(len(tweets)):
    article = ''
    article += tweets[i]['text']
    full_article.append(str(article))


  my_dict = {'created_at' : [],
            'author_id' : [],
            'text': [],
           'anger' : [],
           'disgust' : [],
            'fear' : [],
           'joy' : [],
           'sadness': [],
           'label': [],
           'score': []
           }
  for i in range(len(full_article)):
    text_emotion = getSentimentTwitter(full_article[i])
    my_dict['created_at'].append(date[i])
    my_dict['author_id'].append(author[i])
    my_dict['text'].append(text[i])

    my_dict['anger'].append(text_emotion['anger'])
    my_dict['disgust'].append(text_emotion['disgust'])
    my_dict['fear'].append(text_emotion['fear'])
    my_dict['joy'].append(text_emotion['joy'])
    my_dict['sadness'].append(text_emotion['sadness'])
    my_dict['label'].append(text_emotion['label'])
    my_dict['score'].append(text_emotion['score'])


    df = pd.DataFrame.from_dict(my_dict)
  return df


### Predicted Range

In [72]:
def getPredictedRange(lr, coin_today_df):
  X = coin_today_df[["anger", "disgust", "fear", "joy", 'sadness']]
  y_hat = lr.predict(X)
  return max(y_hat), min(y_hat)  


### Main

In [73]:
def main(twitter_or_news, coin_symbol, coin_name):
  coin_data_df = getData(twitter_or_news, coin_symbol)

  coin_name_bin = coin_symbol + 'USDT'
  coin_price_df = getPriceData(coin_data_df, coin_name_bin)
  coin_price_data_df = combineSentPrice(coin_data_df, coin_price_df)

  lr = dataAnalysis(coin_price_data_df)
  coin_today_df = getTwitterSentToday(coin_name)

  range = getPredictedRange(lr, coin_today_df)

  return range

# Test

In [78]:
main('twitter','XRP','XRP')

ValueError: Found array with 0 sample(s) (shape=(0, 5)) while a minimum of 1 is required.

In [75]:
main('news','ETH','ethereum')

AttributeError: 'NoneType' object has no attribute 'drop'